In [ ]:
import pandas as pd

# We can use the read_excel method to read data from an Excel file
dataframe = pd.read_excel('files/P6-SuperStoreUS-2015.xlsx')

# Let's check out the first 5 rows from our excel file
dataframe.head()

In [ ]:
# We can check out the length of the dataframe
len(dataframe)

In [ ]:
import sqlite3

# Create a connection to a local database (This will create a file called mydatabase.db)
connection = sqlite3.connect('mydatabase.db')

# Copy the dataframe to our SQLite database
dataframe.to_sql('mytable', connection, if_exists='replace')

In [ ]:
import os
from groq import Groq
import dotenv

# Load API key from the .env file
dotenv.load_dotenv()

# Fetch the API key
api_key = os.getenv('GROQ_API_KEY')

# Ensure the API key was loaded properly
if api_key is None:
    raise ValueError("API key not found! Check your .env file and environment variables.")

# Initialize the Groq client with the API key
client = Groq(api_key=api_key)

# Example chat completion request to Groq
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "are the universals real and distinct entities or are they mental constructs?",
        }
    ],
    model="llama3-8b-8192",  # Specify the model name supported by Groq
)

# Output the result
print(chat_completion.choices[0].message.content)



In [82]:
import google.generativeai as genai
import dotenv
import os

# Load API key from the .env file
dotenv.load_dotenv()

# Fetch the API key
api_key = os.getenv('GEMINI_API_KEY')

# Ensure the API key was loaded properly
if api_key is None:
    raise ValueError("API key not found! Check your .env file and environment variables.")

# Configure the Gemini client with the API key
genai.configure(api_key=api_key)

# Initialize the Generative Model
gemini = genai.GenerativeModel("gemini-1.5-flash")


In [ ]:
# Test out the model by generating some text
text = gemini.generate_content("are universals real and distinct entities or merely mental constructs?").text
print(text)

In [83]:
# The doc string is important, as it is passed to the LLM as a description of the function
def sql_query(query: str):
    """Run a SQL SELECT query on a SQLite database and return the results."""
    return pd.read_sql_query(query, connection).to_dict(orient='records')

In [85]:
system_prompt = """
You are an expert SQL analyst. When appropriate, generate SQL queries based on the user question and the database schema.
When you generate a query, use the 'sql_query' function to execute the query on the database and get the results.
Then, use the results to answer the user's question.

database_schema: [
    {
        table: 'mytable',
        columns: [
            {
                name: 'Row ID', l
                type: 'int'
            },
            {
                name: 'Order Priority',
                type: 'string'
            },
            {
                name: 'Discount',
                type: 'float'
            },
            {
                name: 'Unit Price',
                type: 'float'
            },
            {
                name: 'Shipping Cost',
                type: 'float'
            },
            {
                name: 'Customer ID',
                type: 'int'
            },
            {
                name: 'Customer Name',
                type: 'string'
            },
            {
                name: 'Ship Mode',
                type: 'string'
            },
            {
                name: 'Customer Segment',
                type: 'string'
            },
            {
                name: 'Product Category',
                type: 'string'
            },
            {
                name: 'Product Sub-Category',
                type: 'string'
            },
            {
                name: 'Product Container',
                type: 'string'
            },
            {
                name: 'Product Name',
                type: 'string'
            },
            {
                name: 'Product Base Margin',
                type: 'float'
            },
            {
                name: 'Country',
                type: 'string'
            },
            {
                name: 'Region',
                type: 'string'
            },
            {
                name: 'State or Province',
                type: 'string'
            },
            {
                name: 'City',
                type: 'string'
            },
            {
                name: 'Postal Code',
                type: 'string'
            },
            {
                name: 'Order Date',
                type: 'datetime'
            },
            {
                name: 'Ship Date',
                type: 'datetime'
            },
            {
                name: 'Profit',
                type: 'float'
            },
            {   
                name: 'Quantity ordered new',
                type: 'int' 
            },
            {
                name: 'Sales',
                type: 'float'
            },
            {
                name: 'Order ID',
                type: 'int'   
            }           
        ]
    }
]
""".strip() # Call strip to remove leading/trailing whitespace

In [86]:
# Let's create a new Gemini instance with the SQL query function and the system prompt
sql_gemini = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=[sql_query], system_instruction=system_prompt)

In [ ]:
# We begin our chat with Gemini and allow it to use tools when needed
chat = sql_gemini.start_chat(enable_automatic_function_calling=True)

# Let's ask our first question
chat.send_message("which mode of shipping is most used").text

In [ ]:
chat.send_message("which product category has the highest unit price?").text

In [ ]:
chat.send_message("which region bring in the most profit?").text